In [4]:
###### add interleaved gate into rb sequence and add inverse
from functools import reduce
import itertools
import numpy as np
import pickle
import os
from print_large import _print_big_matrix
from tqdm import trange
from multiprocessing import Pool
from tqdm.notebook import tqdm
### 2-qubit Clifford matrix
gateset_path=str(os.path.abspath(os.path.join(os.getcwd(), "../..")))+'/gateset/'
with open(gateset_path+'Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
with open(gateset_path+'3162_rb_seq.pkl', 'rb') as f:
    rb_seq = pickle.load(f)
def find_index(_j):
    if np.isclose(abs(np.trace(interleaved_gate@Cliff_kruas[_j].conj().T))/4,1):
        return _j
def add_irb_inverse(_rb_seq):
    _irb_seq=list(itertools.chain(*zip(_rb_seq, [interleaved_gate_index]*len(_rb_seq))))
    total_operation=reduce(lambda x,y: y@x, [Cliff_kruas[_i] for _i in _irb_seq])
### find irb inverse element
    for i_irb in range(11520):
        if (np.isclose(abs(np.trace(Cliff_kruas[i_irb]@total_operation)),4)):
            _irb_seq.append(i_irb)
            return _irb_seq
        elif i_irb == 11519:
            return False
if __name__ == '__main__':
    irb_seq={}
###interleaved_ part#######################################################################################################################
#### interleaved_gate
    X_2 = 0.5* np.asarray([[1+1j,   1-1j], [1-1j,  1+1j]])
    interleaved_gate= np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
#### find interleaved_gate index    
    pool = Pool()
    interleaved_gate_index=[i for i in filter(lambda x: x!=None, pool.map_async(find_index,[_j for _j in range(11520)]).get())][0]
    pool.close()
    pool.join()
    print('interleaved_gate_index=',interleaved_gate_index)
#######################################################################################################
    for i in tqdm(rb_seq):
        pool=Pool()
        irb_seq[i]=pool.map(add_irb_inverse,[rb_seq[i][_j][:-1] for _j in range(len(rb_seq[i]))])
        assert False not in irb_seq[i], 'false in irbseq'
        pool.close()
        pool.join()
    with open(str(interleaved_gate_index)+'_3162_irb_seq.pkl', 'wb') as f:
        pickle.dump(irb_seq, f)

interleaved_gate_index= 4


  0%|          | 0/33 [00:00<?, ?it/s]

In [2]:
#rb seq w/ inverse
from functools import reduce
import numpy as np
import pickle
import os
from print_large import _print_big_matrix
from tqdm.notebook import tqdm
from multiprocessing import  Pool
a=[1,2,3,4,5,6, 8, 10, 13, 16, 20, 25, 32, 40, 50, 63, 79, 100, 126, 158, 200, 251, 316, 398, 501, 631, 794, 1000, 1259, 1585, 1995, 2512, 3162]

### 2-qubit Clifford matrix
gateset_path=str(os.path.abspath(os.path.join(os.getcwd(), "../..")))+'/gateset/'
with open(gateset_path+'Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
###generate rb sequence
def rb_seq_fun(seq_number):
    np.random.seed()
### random choose 11520 Clifford matrix
    rb_seq_index= np.random.randint(low=0, high=11520, size=seq_number).tolist()
### rb sequence matrix
    rb_seq_kraus=reduce(lambda x, y : y@x, [Cliff_kruas[_i] for _i in rb_seq_index])
### find rb inverse element
    for i_rb in range(11520):
        if (np.isclose(abs(np.trace(Cliff_kruas[i_rb]@rb_seq_kraus)),4)):
            rb_seq_index.append(i_rb)
            return rb_seq_index
        elif i_rb == 11519:
            return False
if __name__ == '__main__':
    rb_seq={}
    max_seq_number=int(input('max sequence number'))
    types=int(input('different types sequence '))
    for i in tqdm(a):
        pool=Pool()
        rb_seq[i]=pool.map(rb_seq_fun,[i]*types)
        assert False not in rb_seq[i], 'false in rbseq'
        pool.close()
        pool.join()
    with open('3162_rb_seq.pkl', 'wb') as f:
        pickle.dump(rb_seq, f)
        

max sequence number1
different types sequence 1000


  0%|          | 0/33 [00:00<?, ?it/s]